# نظام تقدير استهلاك الطعام بدقة

دفتر جوبيتر هذا يستخدم كود `PrecisionFoodDetector` لتحليل كمية الطعام المستهلك من صور قبل وبعد الأكل.

🔸 يمكنك تعديل المسارات واستخدام صورك الخاصة.

**المكتبات المطلوبة:**
- opencv-python
- matplotlib
- scikit-learn
- numpy

استخدم الأمر التالي للتثبيت (إذا لزم الأمر):
```bash
!pip install opencv-python matplotlib scikit-learn numpy
```

In [ ]:
# استيراد المكتبات
import warnings
warnings.filterwarnings("ignore")

from typing import Dict, Any, Optional, Tuple
import cv2
import numpy as np
import matplotlib.pyplot as plt
from sklearn.mixture import GaussianMixture

## تعريف الكلاس الرئيسي
كود النظام كاملًا كما هو في ملفك البرمجي.

In [ ]:
class PrecisionFoodDetector:
    """
    نظام تقدير استهلاك الطعام باستخدام الرؤية الحاسوبية التقليدية (بدون شبكات عميقة).

    المراحل:
      1) Preprocessing: Bilateral + CLAHE(LAB)
      2) Plate/Container mask: Edges/Contours (+ fallback)
      3) Segmentation: Gaussian Mixture (GMM) على ميزات لونية + مكانية
      4) Refinement: عمليات مورفولوجية + إزالة المكوّنات الصغيرة
      5) قياس المساحة ثم حساب النِّسَب قبل/بعد

    Parameters
    ----------
    debug_mode : bool
        لطباعة معلومات أثناء التنفيذ.
    """

    def __init__(self, debug_mode: bool = True) -> None:
        self.debug_mode = debug_mode

    def load_and_preprocess(self, image_path: str) -> np.ndarray:
        img = cv2.imread(image_path)
        if img is None:
            raise ValueError(f"Cannot load image: {image_path}")
        img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        h, w = img_rgb.shape[:2]
        if w > 800:
            scale = 800 / w
            img_rgb = cv2.resize(img_rgb, (int(w * scale), int(h * scale)), interpolation=cv2.INTER_AREA)
        return img_rgb

    def advanced_preprocessing(self, img_rgb: np.ndarray) -> np.ndarray:
        img_float = img_rgb.astype(np.float32) / 255.0
        filtered_u8 = cv2.bilateralFilter((img_float * 255).astype(np.uint8), d=9, sigmaColor=75, sigmaSpace=75)
        lab = cv2.cvtColor(filtered_u8, cv2.COLOR_RGB2LAB)
        clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
        lab[:, :, 0] = clahe.apply(lab[:, :, 0])
        enhanced = cv2.cvtColor(lab, cv2.COLOR_LAB2RGB)
        return enhanced

    def detect_container_boundary(self, img_rgb: np.ndarray) -> np.ndarray:
        gray = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2GRAY)
        edges_fine = cv2.Canny(gray, 50, 150, apertureSize=3)
        edges_coarse = cv2.Canny(gray, 30, 100, apertureSize=5)
        edges = cv2.bitwise_or(edges_fine, edges_coarse)
        edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)))
        contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        h, w = gray.shape[:2]
        mask = np.zeros((h, w), dtype=np.uint8)
        if contours:
            largest = max(contours, key=cv2.contourArea)
            cv2.fillPoly(mask, [largest], 255)
            mask = cv2.erode(mask, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (15, 15)), iterations=1)
            return mask
        cv2.ellipse(mask, (w // 2, h // 2), (w // 3, h // 3), 0, 0, 360, 255, -1)
        return mask

    def gaussian_mixture_segmentation(self, img_rgb: np.ndarray, container_mask: np.ndarray) -> np.ndarray:
        h, w = img_rgb.shape[:2]
        if container_mask is None or cv2.countNonZero(container_mask) == 0:
            return np.zeros((h, w), dtype=np.uint8)
        hsv = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2HSV)
        lab = cv2.cvtColor(img_rgb, cv2.COLOR_RGB2LAB)
        ys, xs = np.where(container_mask > 0)
        if len(xs) == 0:
            return np.zeros((h, w), dtype=np.uint8)
        rgb_feats = img_rgb[ys, xs].astype(np.float32) / 255.0
        hsv_feats = hsv[ys, xs].astype(np.float32) / np.array([180.0, 255.0, 255.0], dtype=np.float32)
        lab_feats = lab[ys, xs].astype(np.float32) / np.array([100.0, 255.0, 255.0], dtype=np.float32)
        spatial = np.stack([xs / float(w), ys / float(h)], axis=1).astype(np.float32)
        features = np.concatenate([rgb_feats, hsv_feats, lab_feats, spatial], axis=1)
        n_components = int(min(8, max(2, features.shape[0] // 100 + 2)))
        gmm = GaussianMixture(n_components=n_components, covariance_type="full", random_state=42)
        labels = gmm.fit_predict(features)
        food_mask = np.zeros((h, w), dtype=np.uint8)
        for cid in range(n_components):
            cluster_idx = (labels == cid)
            if not np.any(cluster_idx):
                continue
            cluster_feats = features[cluster_idx]
            cluster_pos = np.stack([ys[cluster_idx], xs[cluster_idx]], axis=1)
            if self.analyze_cluster_for_food(cluster_feats):
                food_mask[cluster_pos[:, 0], cluster_pos[:, 1]] = 255
        return food_mask

    def analyze_cluster_for_food(self, cluster_features: np.ndarray) -> bool:
        if cluster_features.size == 0:
            return False
        rgb = (cluster_features[:, 0:3] * 255.0)
        hsv = (cluster_features[:, 3:6] * np.array([180.0, 255.0, 255.0], dtype=np.float32))
        mean_sat = float(np.mean(hsv[:, 1]))
        mean_val = float(np.mean(hsv[:, 2]))
        mean_rgb_std = float(np.mean(np.std(rgb, axis=0)))
        cluster_size_ok = (len(cluster_features) >= 200)
        is_food = (
            mean_sat > 25 and
            20 < mean_val < 220 and
            mean_rgb_std > 5 and
            cluster_size_ok
        )
        return is_food

    def refine_with_morphology(self, mask: np.ndarray, reference_img: np.ndarray) -> np.ndarray:
        if mask is None or mask.size == 0:
            return mask
        mask = cv2.morphologyEx(mask, cv2.MORPH_OPEN, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (3, 3)))
        mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (7, 7)))
        num_labels, labels, stats, _ = cv2.connectedComponentsWithStats(mask)
        total_pixels = reference_img.shape[0] * reference_img.shape[1]
        min_area = max(1, int(total_pixels * 0.001))
        refined = np.zeros_like(mask)
        for i in range(1, num_labels):
            if stats[i, cv2.CC_STAT_AREA] >= min_area:
                refined[labels == i] = 255
        return refined

    def detect_food_in_image(self, image_path: str) -> Optional[Dict[str, Any]]:
        try:
            img = self.load_and_preprocess(image_path)
            enhanced = self.advanced_preprocessing(img)
            if self.debug_mode:
                print(f"✓ Loaded image: {img.shape}")
            container_mask = self.detect_container_boundary(enhanced)
            if self.debug_mode:
                print(f"✓ Container detected: {int(np.sum(container_mask > 0))} pixels")
            food_mask_raw = self.gaussian_mixture_segmentation(enhanced, container_mask)
            if self.debug_mode:
                print(f"✓ Initial food detection: {int(np.sum(food_mask_raw > 0))} pixels")
            food_mask = self.refine_with_morphology(food_mask_raw, img)
            food_area = int(np.sum(food_mask > 0))
            if self.debug_mode:
                print(f"✓ Final food area: {food_area} pixels")
            return {
                "original_image": img,
                "enhanced_image": enhanced,
                "container_mask": container_mask,
                "food_mask": food_mask,
                "food_area": food_area,
            }
        except Exception as e:
            print(f"❌ Error processing {image_path}: {e}")
            return None

    def calculate_consumption(self, before_path: str, after_path: str) -> Optional[Dict[str, Any]]:
        print(" PRECISION FOOD DETECTION SYSTEM")
        print("=" * 50)
        print("\n Processing BEFORE image...")
        before = self.detect_food_in_image(before_path)
        if before is None:
            print("❌ Failed to process BEFORE image")
            return None
        print("\n Processing AFTER image...")
        after = self.detect_food_in_image(after_path)
        if after is None:
            print("❌ Failed to process AFTER image")
            return None
        before_area = before["food_area"]
        after_area = after["food_area"]
        if before_area > 0:
            consumed_pct = ((before_area - after_area) / before_area) * 100.0
            remaining_pct = (after_area / before_area) * 100.0
        else:
            consumed_pct = 0.0
            remaining_pct = 0.0
        consumed_pct = float(np.clip(consumed_pct, 0.0, 100.0))
        remaining_pct = float(np.clip(remaining_pct, 0.0, 100.0))
        return {
            "consumed_percentage": consumed_pct,
            "remaining_percentage": remaining_pct,
            "before_area": int(before_area),
            "after_area": int(after_area),
            "before_result": before,
            "after_result": after,
        }

    def visualize_results(self, results: Dict[str, Any]) -> None:
        fig, axes = plt.subplots(3, 4, figsize=(20, 15))
        before = results["before_result"]
        after = results["after_result"]
        axes[0, 0].imshow(before["original_image"]); axes[0, 0].set_title("Before - Original", fontweight="bold"); axes[0, 0].axis("off")
        axes[0, 1].imshow(before["enhanced_image"]); axes[0, 1].set_title("Before - Enhanced", fontweight="bold"); axes[0, 1].axis("off")
        axes[0, 2].imshow(after["original_image"]);  axes[0, 2].set_title("After - Original",  fontweight="bold"); axes[0, 2].axis("off")
        axes[0, 3].imshow(after["enhanced_image"]);  axes[0, 3].set_title("After - Enhanced",  fontweight="bold"); axes[0, 3].axis("off")
        axes[1, 0].imshow(before["container_mask"], cmap="gray"); axes[1, 0].set_title("Before - Container", fontweight="bold"); axes[1, 0].axis("off")
        axes[1, 1].imshow(before["food_mask"], cmap="gray");      axes[1, 1].set_title("Before - Food Detection", fontweight="bold"); axes[1, 1].axis("off")
        axes[1, 2].imshow(after["container_mask"], cmap="gray");  axes[1, 2].set_title("After - Container",  fontweight="bold"); axes[1, 2].axis("off")
        axes[1, 3].imshow(after["food_mask"], cmap="gray");       axes[1, 3].set_title("After - Food Detection",  fontweight="bold"); axes[1, 3].axis("off")
        overlay_b = before["original_image"].copy()
        overlay_b[before["food_mask"] == 0] = overlay_b[before["food_mask"] == 0] // 2
        axes[2, 0].imshow(overlay_b); axes[2, 0].set_title("Before - Food Highlighted", fontweight="bold"); axes[2, 0].axis("off")
        overlay_a = after["original_image"].copy()
        overlay_a[after["food_mask"] == 0] = overlay_a[after["food_mask"] == 0] // 2
        axes[2, 1].imshow(overlay_a); axes[2, 1].set_title("After - Remaining Food", fontweight="bold"); axes[2, 1].axis("off")
        if before["food_mask"].shape == after["food_mask"].shape:
            diff = cv2.absdiff(before["food_mask"], after["food_mask"])
            axes[2, 2].imshow(diff, cmap="Reds"); axes[2, 2].set_title("Consumed Areas", fontweight="bold"); axes[2, 2].axis("off")
        else:
            axes[2, 2].text(0.5, 0.5, "Different sizes", ha="center", va="center"); axes[2, 2].axis("off")
        axes[2, 3].text(0.1, 0.8, "🍽️ CONSUMED", fontsize=16, fontweight="bold", transform=axes[2, 3].transAxes)
        axes[2, 3].text(0.1, 0.65, f"{results['consumed_percentage']:.1f}%", fontsize=28, color="red", fontweight="bold", transform=axes[2, 3].transAxes)
        axes[2, 3].text(0.1, 0.4, "📦 REMAINING", fontsize=16, fontweight="bold", transform=axes[2, 3].transAxes)
        axes[2, 3].text(0.1, 0.25, f"{results['remaining_percentage']:.1f}%", fontsize=28, color="green", fontweight="bold", transform=axes[2, 3].transAxes)
        axes[2, 3].axis("off")
        plt.suptitle(" PRECISION FOOD DETECTION RESULTS", fontsize=18, fontweight="bold")
        plt.tight_layout()
        plt.show()

## دوال المساعدة لتحليل الصور

In [ ]:
def precision_food_analysis(before_image: str, after_image: str) -> Optional[float]:
    detector = PrecisionFoodDetector(debug_mode=True)
    print("Starting PRECISION FOOD DETECTION")
    print("Using: Gaussian Mixture Model + Advanced Computer Vision")
    results = detector.calculate_consumption(before_image, after_image)
    if results:
        detector.visualize_results(results)
        return results["consumed_percentage"]
    print("❌ Analysis failed")
    return None

def test_single_image(image_path: str) -> Optional[Dict[str, Any]]:
    detector = PrecisionFoodDetector(debug_mode=True)
    result = detector.detect_food_in_image(image_path)
    if result:
        plt.figure(figsize=(15, 5))
        plt.subplot(1, 3, 1); plt.imshow(result["original_image"]); plt.title("Original"); plt.axis("off")
        plt.subplot(1, 3, 2); plt.imshow(result["food_mask"], cmap="gray"); plt.title("Food Mask (White=Food)"); plt.axis("off")
        overlay = result["original_image"].copy()
        overlay[result["food_mask"] == 0] = [255, 0, 0]
        plt.subplot(1, 3, 3); plt.imshow(cv2.addWeighted(result["original_image"], 0.7, overlay, 0.3, 0))
        plt.title("Food Highlighted"); plt.axis("off")
        plt.suptitle(f'Food Area: {result["food_area"]:,} pixels')
        plt.show()
        return result
    return None

## تجربة النظام

عدل مسارات الصور حسب ملفاتك، ثم نفذ الخلية أدناه لتحليل الاستهلاك:

In [ ]:
# مثال: ضع مسارات الصور هنا
before_path = "image_before.jpg"  # صورة قبل الأكل
after_path  = "image_after.jpg"   # صورة بعد الأكل

consumption = precision_food_analysis(before_path, after_path)
if consumption is not None:
    print(f"\n✅ FINAL RESULT: {consumption:.1f}% of food consumed")
else:
    print("\n❌ Could not complete analysis")